In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from hazm import Normalizer
from fuzzywuzzy import fuzz
import re
from sklearn.preprocessing import MinMaxScaler


c:\Users\MohammadRasoul Azizi\anaconda3\envs\hazm_env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from hazm import Normalizer, word_tokenize, stopwords_list
from nltk.tokenize import word_tokenize as nltk_tokenize
from nltk.corpus import stopwords
import nltk
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix


In [4]:
# Download the data required by NLTK
try:
    nltk.data.find('corpora/stopwords')
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("NLTK resources not found. Downloading...")
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('punkt_tab')
    print("NLTK resources downloaded.")


# ۱. بارگذاری و پیش‌پردازش اولیه داده‌ها

In [5]:
df=pd.read_csv("../data/searching_job.csv", encoding='utf-8')

In [6]:
df.head()

,ID,Applicant ID,Full Name,Source,Order,Developer State,Seen,Group Apply Source Job ID,Job ID,Careers Job - Job → Title,...,Careers Job - Job → Requirements,Careers Job - Job → Level,Careers Job - Job → Offers Remote,Careers Job - Job → Minimum Salary,Careers Job - Job → State,Careers Job - Job → Maximum Salary,Careers Job - Job → Hit Count,Accounts City - City → Name,Careers Disqualifyreason - Disqualify Reason → Name,Careers Job - Group Apply Source Job → Title
0,393696,85372.0,D D,2,164,10,False,16487.0,20092,Back-end Developer (Python),...,"<p dir=""ltr""><strong>Minimum requirements </st...",junior,True,"7,000,000",C,"12,000,000","2,125",تهران,NaN,توسعه‌دهنده (Back-end (Python
1,380461,149261.0,Z T,2,0,25,True,16487.0,19784,توسعه‌دهنده Back-end,...,"<p dir=""rtl""><strong>مهارت‌های ضروری:</strong>...",senior,False,"12,000,000",C,"16,000,000","1,106",تهران,NaN,توسعه‌دهنده (Back-end (Python
2,380462,149261.0,X H,2,5,30,True,16487.0,19718,senior Back-end Developer (Django),...,"<p dir=""auto""><strong>مهارت‌های ضروری:</strong...",senior,False,"25,000,000",C,"30,000,000","1,055",تهران,مهارت یا تجربه ناکافی,توسعه‌دهنده (Back-end (Python
3,393698,85372.0,J U,2,0,30,True,16487.0,19916,Senior Back-end Developer (Python),...,"<p dir=""auto""><strong>We expect you to be an e...",senior,True,"35,000,000",A,"40,000,000","1,405",تهران,مهارت یا تجربه ناکافی,توسعه‌دهنده (Back-end (Python
4,380459,149261.0,G C,2,2,30,True,16487.0,19985,برنامه‌نویس Back-end وب,...,"<p dir=""auto""><strong>مهارت ‌ها و شرایط فنی:</...",senior,False,NaN,C,NaN,530,تهران,مهارت یا تجربه ناکافی,توسعه‌دهنده (Back-end (Python


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15454 entries, 0 to 15453
Data columns (total 21 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   ID                                                   15454 non-null  int64  
 1   Applicant ID                                         15454 non-null  float64
 2   Full Name                                            15134 non-null  object 
 3   Source                                               15454 non-null  int64  
 4   Order                                                15454 non-null  int64  
 5   Developer State                                      15454 non-null  int64  
 6   Seen                                                 15454 non-null  bool   
 7   Group Apply Source Job ID                            4163 non-null   float64
 8   Job ID                                               15454 non-nul

In [8]:
df.isnull().sum() / len(df) * 100

ID                                                      0.000000
Applicant ID                                            0.000000
Full Name                                               2.070661
Source                                                  0.000000
Order                                                   0.000000
Developer State                                         0.000000
Seen                                                    0.000000
Group Apply Source Job ID                              73.061990
Job ID                                                  0.000000
Careers Job - Job → Title                               0.000000
Careers Job - Job → Description                         0.000000
Careers Job - Job → Requirements                        0.304128
Careers Job - Job → Level                               0.000000
Careers Job - Job → Offers Remote                       0.000000
Careers Job - Job → Minimum Salary                     15.808205
Careers Job - Job → State

## تبدیل Seen به مقادیر عددی


In [9]:
df['Seen'] = df['Seen'].replace({True: 1, False: 0})

C:\Users\MohammadRasoul Azizi\AppData\Local\Temp\ipykernel_27572\1248386997.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Seen'] = df['Seen'].replace({True: 1, False: 0})


In [10]:
df.Seen.value_counts()

Seen
1    15048
0      406
Name: count, dtype: int64

## مدیریت داده‌های گمشده

In [11]:
df['Careers Job - Job → Minimum Salary'] = df['Careers Job - Job → Minimum Salary'].str.replace(',', '').astype(float, errors='ignore').fillna(0)
df['Careers Job - Job → Maximum Salary'] = df['Careers Job - Job → Maximum Salary'].str.replace(',', '').astype(float, errors='ignore').fillna(0)
df['Careers Job - Job → Hit Count'] = df['Careers Job - Job → Hit Count'].str.replace(',', '').astype(float)
df['Accounts City - City → Name'] = df['Accounts City - City → Name'].fillna('نامشخص')
df['Careers Disqualifyreason - Disqualify Reason → Name'] = df['Careers Disqualifyreason - Disqualify Reason → Name'].fillna('نامشخص')

In [12]:
df.isnull().sum() / len(df) * 100

ID                                                      0.000000
Applicant ID                                            0.000000
Full Name                                               2.070661
Source                                                  0.000000
Order                                                   0.000000
Developer State                                         0.000000
Seen                                                    0.000000
Group Apply Source Job ID                              73.061990
Job ID                                                  0.000000
Careers Job - Job → Title                               0.000000
Careers Job - Job → Description                         0.000000
Careers Job - Job → Requirements                        0.304128
Careers Job - Job → Level                               0.000000
Careers Job - Job → Offers Remote                       0.000000
Careers Job - Job → Minimum Salary                      0.000000
Careers Job - Job → State

In [13]:
threshold = 0.5  # آستانه‌ی ۵۰ درصد

# # محاسبه نسبت داده‌های غیر گمشده در هر ستون و نگه‌داشتن فقط ستون‌هایی که بیش از ۵۰٪ داده دارند
df = df.loc[:, df.isnull().mean() <= threshold]

In [14]:
df.isnull().sum() / len(df) * 100

ID                                                     0.000000
Applicant ID                                           0.000000
Full Name                                              2.070661
Source                                                 0.000000
Order                                                  0.000000
Developer State                                        0.000000
Seen                                                   0.000000
Job ID                                                 0.000000
Careers Job - Job → Title                              0.000000
Careers Job - Job → Description                        0.000000
Careers Job - Job → Requirements                       0.304128
Careers Job - Job → Level                              0.000000
Careers Job - Job → Offers Remote                      0.000000
Careers Job - Job → Minimum Salary                     0.000000
Careers Job - Job → State                              0.000000
Careers Job - Job → Maximum Salary      

## نرمال‌سازی Hit Count

In [15]:
scaler = MinMaxScaler()
df['Hit Count Norm'] = scaler.fit_transform(df[['Careers Job - Job → Hit Count']])

## بررسی داده‌ها

In [17]:
print(f"تعداد کاربران یکتا: {df['Applicant ID'].nunique()}")
print(f"تعداد شغل‌های یکتا: {df['Job ID'].nunique()}")

تعداد کاربران یکتا: 5702
تعداد شغل‌های یکتا: 1783


# ۲. نرمال‌سازی ستون Title

In [18]:
def normalize_title(title, normalizer):
    if pd.isna(title):
        return ""
    # نرمال‌سازی متن فارسی
    title = normalizer.normalize(title)
    # حذف علائم نگارشی و اعداد
    title = re.sub(r'[۰-۹0-9]|[^\w\s]', '', title)
    # تبدیل به حروف کوچک برای انگلیسی
    title = title.lower()
    return title

# ۳. استخراج تکنولوژی و حوزه شغلی

In [19]:

technologies = {
    'python': ['پایتون', 'python', 'py'],
    'javascript': ['جاوااسکریپت', 'javascript', 'js'],
    'react': ['ری‌اکت', 'react'],
    'django': ['جنگو', 'django'],
    'java': ['جاوا', 'java'],
    'sql': ['اس‌کیوال', 'sql'],
    'node': ['نود', 'node', 'nodejs'],
    'angular': ['انگولار', 'angular'],
    'typescript': ['تایپ‌اسکریپت', 'typescript', 'ts'],
}

job_domains = {
    'frontend': ['فرانت‌اند', 'frontend', 'front-end'],
    'backend': ['بک‌اند', 'backend', 'back-end'],
    'data_science': ['دیتا ساینس', 'data science', 'داده‌کاوی'],
    'blockchain': ['بلاک‌چین', 'blockchain'],
    'ai': ['هوش مصنوعی', 'ai', 'artificial intelligence'],
    'devops': ['دواپس', 'devops'],
    'fullstack': ['فول‌استک', 'fullstack', 'full-stack'],
}

def extract_features_from_title(title):
    if pd.isna(title):
        return '', ''
    
    title = title.lower()
    tech_found = []
    domain_found = []
    
    # استخراج تکنولوژی‌ها
    for tech, aliases in technologies.items():
        for alias in aliases:
            if fuzz.partial_ratio(alias, title) > 80 or alias in title:
                tech_found.append(tech)
                break
    
    # استخراج حوزه‌های شغلی
    for domain, aliases in job_domains.items():
        for alias in aliases:
            if fuzz.partial_ratio(alias, title) > 80 or alias in title:
                domain_found.append(domain)
                break
    
    return ', '.join(set(tech_found)), ', '.join(set(domain_found))

## نرمال‌سازی عناوین

In [20]:
normalizer = Normalizer()
df['normalized_title'] = df['Careers Job - Job → Title'].apply(lambda x: normalize_title(x, normalizer))

In [21]:
df["normalized_title"].head()

0           backend developer python
1                 توسعهدهنده backend
2    senior backend developer django
3    senior backend developer python
4              برنامهنویس backend وب
Name: normalized_title, dtype: object

## استخراج تکنولوژی و حوزه شغلی

In [22]:
df[['technologies', 'job_domain']] = df['Careers Job - Job → Title'].apply(
    lambda x: pd.Series(extract_features_from_title(x))
)

In [32]:
df[['technologies', 'job_domain']].sample(10)

,technologies,job_domain
6845,,backend
7065,,
6798,react,frontend
15162,react,
6514,,
1749,,frontend
5706,,
12319,,
5984,,
11166,,backend


## ایجاد presentation_content

In [30]:
df['presentation_content'] = ( 
    df['job_domain'].fillna('') + " " +
    df['technologies'].fillna('') + " " + 
    df["normalized_title"]   
)

In [33]:
df['presentation_content'].sample(10)

4232                            کارآموز c net framework
5779     backend python senior backend developer python
10319         django django internship remote or hybrid
4610            backend python python developer backend
13064                     frontend  توسعهدهنده frontend
8069          django django internship remote or hybrid
11111         django django internship remote or hybrid
8770                     frontend  توسعهدهنده front end
9517      django, python توسعهدهنده ارشد python  django
10687         django django internship remote or hybrid
Name: presentation_content, dtype: object

# ۴. تحلیل محتوا-محور

In [34]:
def content_based_filtering(df, top_k=10):
    stop_words = ['و', 'در', 'به', 'از', 'این', 'که', 'the', 'is', 'a', 'with', 'for']
    vectorizer = TfidfVectorizer(max_features=1000, stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(df['presentation_content'])
    similarity_matrix_content = cosine_similarity(tfidf_matrix)
    
    return similarity_matrix_content, tfidf_matrix, vectorizer

# ۵. فیلترینگ مشارکتی

In [35]:
def collaborative_filtering(df):
    # ایجاد ماتریس تعاملات
    interaction_matrix = df.pivot_table(
        index='Applicant ID', columns='Job ID', values='Seen', fill_value=0
    )
    interaction_matrix_sparse = csr_matrix(interaction_matrix.values)
    similarity_matrix_collab = cosine_similarity(interaction_matrix_sparse.T)
    
    return similarity_matrix_collab, interaction_matrix  # interaction_matrix به صورت DataFrame

# ۶. فیلترهای سخت

In [36]:
def apply_hard_filters(df, applicant_prefs, job_ids):
    filtered_df = df[df['Job ID'].isin(job_ids)].copy()
    
    # فیلتر سطح شغل
    if 'level' in applicant_prefs:
        filtered_df = filtered_df[filtered_df['Careers Job - Job → Level'] == applicant_prefs['level']]
    
    # فیلتر شهرها
    if 'cities' in applicant_prefs:
        filtered_df = filtered_df[filtered_df['Accounts City - City → Name'].isin(applicant_prefs['cities'])]
    
    # فیلتر دورکاری
    if 'remote' in applicant_prefs:
        filtered_df = filtered_df[filtered_df['Careers Job - Job → Offers Remote'] == applicant_prefs['remote']]
    
    # فیلتر حداقل حقوق
    if 'min_salary' in applicant_prefs:
        filtered_df = filtered_df[filtered_df['Careers Job - Job → Minimum Salary'] >= applicant_prefs['min_salary']]
    
    print(f"تعداد شغل‌های مطابق با فیلترها: {len(filtered_df)}")
    return filtered_df['Job ID'].tolist()

# ۷. سیستم پیشنهاددهنده هیبریدی

In [48]:
# 7. سیستم پیشنهاددهنده هیبریدی
def hybrid_recommendation(df, user_id, applicant_prefs, similarity_matrix_content, 
                         similarity_matrix_collab, interaction_matrix, top_k=5, content_weight=0.5):
    job_ids = df['Job ID'].unique()
    
    # بررسی Cold Start
    if user_id not in interaction_matrix.index:
        print(f"کاربر {user_id} در ماتریس تعاملات وجود ندارد. استفاده از محبوبیت شغل‌ها.")
        recommendations = df.groupby('Job ID').agg({
            'Hit Count Norm': 'mean',
            'technologies': 'first',
            'job_domain': 'first',
            'Careers Job - Job → Title': 'first'
        }).reset_index()
        recommendations = recommendations[recommendations['Job ID'].isin(
            apply_hard_filters(df, applicant_prefs, job_ids)
        )]
        # حذف تکراری‌ها
        recommendations = recommendations.drop_duplicates(subset=['Job ID'])
        # مرتب‌سازی بر اساس محبوبیت و تنوع محتوایی
        recommendations['content_score'] = recommendations.index.map(
            lambda idx: similarity_matrix_content[idx].mean() if idx < similarity_matrix_content.shape[0] else 0
        )
        recommendations['final_score'] = 0.7 * recommendations['Hit Count Norm'] + 0.3 * recommendations['content_score']
        recommendations = recommendations.sort_values('final_score', ascending=False)
        return recommendations[['Job ID', 'Careers Job - Job → Title', 'technologies', 'job_domain']].head(top_k)
    
    # امتیاز محتوا-محور
    content_scores = np.zeros(len(job_ids))
    for idx, job_id in enumerate(job_ids):
        job_idx = df[df['Job ID'] == job_id].index[0]
        content_scores[idx] = similarity_matrix_content[job_idx].mean()
    
    # امتیاز فیلترینگ مشارکتی
    user_idx = interaction_matrix.index.get_loc(user_id)
    collab_scores = np.zeros(len(job_ids))
    for idx, job_id in enumerate(job_ids):
        if job_id in interaction_matrix.columns:
            job_col_idx = interaction_matrix.columns.get_loc(job_id)
            collab_scores[idx] = similarity_matrix_collab[job_col_idx].mean()
    
    # ترکیب امتیازها
    final_scores = content_weight * content_scores + (1 - content_weight) * collab_scores
    
    # اعمال فیلترهای سخت
    valid_job_ids = apply_hard_filters(df, applicant_prefs, job_ids)
    valid_indices = [i for i, job_id in enumerate(job_ids) if job_id in valid_job_ids]
    
    if not valid_job_ids:
        print("هیچ شغلی با فیلترهای مشخص‌شده یافت نشد.")
        return pd.DataFrame(columns=['Job ID', 'Careers Job - Job → Title', 'technologies', 'job_domain'])
    
    # مرتب‌سازی و انتخاب پیشنهادها
    final_scores = final_scores[valid_indices]
    top_indices = np.argsort(final_scores)[::-1][:top_k]
    top_job_ids = [job_ids[valid_indices[i]] for i in top_indices]
    
    recommendations = df[df['Job ID'].isin(top_job_ids)][
        ['Job ID', 'Careers Job - Job → Title', 'technologies', 'job_domain']
    ].drop_duplicates()
    recommendations['Hybrid Score'] = final_scores[top_indices][:len(recommendations)]
    
    return recommendations

In [55]:

# 8. انتخاب کاربر با تعاملات کافی
def select_active_user(df, number_of_users= 5, min_interactions=5):
    user_interactions = df.groupby('Applicant ID')['Seen'].sum()
    active_users = user_interactions[user_interactions >= min_interactions].index
    if len(active_users) == 0:
        print("هیچ کاربری با تعاملات کافی یافت نشد.")
        return None
    return map(int, active_users[0: number_of_users])


# ۸. اجرای اصلی

In [57]:
# انتخاب کاربر فعال
user_id = select_active_user(df)

print(f"\nکاربر انتخاب‌شده: \n{list(user_id)}")


کاربر انتخاب‌شده: 
[2751, 3904, 4005, 5797, 6003]


## اجرای تحلیل محتوا-محور و فیلترینگ مشارکتی

In [ ]:
similarity_matrix_content, tfidf_matrix, vectorizer = content_based_filtering(df)
similarity_matrix_collab, interaction_matrix = collaborative_filtering(df)


In [60]:

# تنظیمات کاربر
user_id = 6003
content_weights = [0, 0.25, 0.5, 0.75, 1]
# applicant_prefs = {
#     'level': 'senior',
#     'cities': ['تهران'],
#     'remote': True,
#     'min_salary': 20000000,
# }

applicant_prefs={}
# ذخیره نتایج
submission = {}
for weight in content_weights:
    print(f"\nوزن محتوا-محور: {weight}")
    recommendations = hybrid_recommendation(
        df, user_id, applicant_prefs, similarity_matrix_content, 
        similarity_matrix_collab, interaction_matrix, top_k=5, content_weight=weight
    )
    submission[f'job_id_{weight}'] = recommendations['Job ID'].tolist()[:5]
    print(recommendations)


وزن محتوا-محور: 0
تعداد شغل‌های مطابق با فیلترها: 15454
      Job ID         Careers Job - Job → Title technologies job_domain  \
1018   20552        توسعه‌دهنده Back-end (رشت)                 backend   
1087   20186   Senior .NET Developer (دورکاری)                           
1786   21367             برنامه‌نویس WordPress                           
1787   21357    برنامه‌نویس بک اند PHP Laravel                 backend   
2953   22516  Back-end) Senior Java Developer)         java    backend   

      Hybrid Score  
1018      0.024206  
1087      0.020458  
1786      0.019607  
1787      0.019607  
2953      0.019607  

وزن محتوا-محور: 0.25
تعداد شغل‌های مطابق با فیلترها: 15454
      Job ID Careers Job - Job → Title technologies job_domain  Hybrid Score
2086   21565     توسعه‌دهنده Front-end                frontend      0.072350
2155   21676     توسعه‌دهنده Front-end                frontend      0.071188
2182   21467     توسعه‌دهنده Front-end                frontend      0.070998
2217